# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import seaborn as sn
import scipy
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data = pd.read_csv('../Output/data/cities_new.csv')
cities_data.dropna()
cities_data.head(20)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Country,Wind Speed,Date
0,0,hermanus,-34.42,19.23,73.99,66,28,ZA,1.99,1605948168
1,1,new norfolk,6.65,-10.59,80.60,88,75,LR,2.10,1605948168
2,2,mataura,-46.19,168.86,48.99,85,100,NZ,13.00,1605948168
3,3,noumea,-22.28,166.46,73.40,69,75,NC,14.99,1605948146
4,4,cabo san lucas,-8.28,-35.03,77.00,73,40,BR,5.82,1605947905
5,5,de-kastri,51.47,140.78,26.46,81,100,RU,5.06,1605948169
6,6,ancud,-41.87,-73.82,54.00,100,90,CL,14.99,1605948169
7,7,merauke,-8.47,140.33,84.74,72,100,ID,18.39,1605948169
8,8,kyzyl-suu,42.34,78.01,33.39,82,9,KG,2.04,1605948170
9,9,nikolskoye,59.70,30.79,34.00,87,75,RU,4.47,1605948170


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
from IPython.display import display
import ipywidgets as widgets
import gmaps

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [16]:
# Store 'Lat' and 'Lng' into  locations
locations = cities_data[["Lat", "Lng"]].astype(float)
humidity = cities_data["Humidity"].astype(float)

# Set dimensions for graph
figure_layout = {
     'width' : '1000px',
     'height':'400px',
     'border':'1px solid black',
     'padding':'1px',
     'margin':'0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
fig = gmaps.figure()

# Create and add heat layer 
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=100,
                                   point_radius=5)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city = cities_data.loc[(cities_data["Max Temp"] > 70) & (cities_data["Max Temp"] < 80) & (cities_data["Cloudiness"] == 0), :]
narrowed_city = narrowed_city.dropna(how ='any')
narrowed_city.reset_index(inplace=True)
del narrowed_city['index']
narrowed_city.head(15)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Country,Wind Speed,Date
0,35,busselton,-33.65,115.33,73.99,44,0,AU,14.41,1605948176
1,80,arlit,18.74,7.39,72.97,21,0,NE,2.77,1605948185
2,101,abu dhabi,24.60,72.70,70.45,22,0,IN,6.73,1605948190
3,152,banda aceh,25.48,80.33,78.49,19,0,IN,11.83,1605948201
4,228,bela,25.93,81.98,77.88,22,0,IN,11.99,1605948218
5,282,oussouye,12.48,-16.55,73.40,94,0,SN,2.24,1605948230
6,401,natitingou,10.30,1.38,78.80,27,0,BJ,9.17,1605948258
7,450,riyadh,24.69,46.72,78.80,26,0,SA,14.99,1605947993
8,495,hermosillo,29.07,-110.97,73.00,36,0,MX,1.01,1605948085
9,506,coahuayana,18.73,-103.68,74.71,80,0,MX,3.36,1605948280


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# create hotel_df with hotel name column
hotel_df = pd.DataFrame(cities_data, columns=['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Hotel Name'])
hotel_df.head(20)

,City,Country,Lat,Lng,Max Temp,Hotel Name
0,hermanus,ZA,-34.42,19.23,73.99,NaN
1,new norfolk,LR,6.65,-10.59,80.60,NaN
2,mataura,NZ,-46.19,168.86,48.99,NaN
3,noumea,NC,-22.28,166.46,73.40,NaN
4,cabo san lucas,BR,-8.28,-35.03,77.00,NaN
5,de-kastri,RU,51.47,140.78,26.46,NaN
6,ancud,CL,-41.87,-73.82,54.00,NaN
7,merauke,ID,-8.47,140.33,84.74,NaN
8,kyzyl-suu,KG,42.34,78.01,33.39,NaN
9,nikolskoye,RU,59.70,30.79,34.00,NaN


In [7]:
hotel_list = []
for i in range(len(narrowed_city)):
    lat = narrowed_city.loc[i]['Lat']
    lng = narrowed_city.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_list.append(jsn['results'][0]['name'])
    except:
        hotel_list.append("")
narrowed_city["Hotel Name"] = hotel_list
narrowed_city = narrowed_city.dropna(how='any')
narrowed_city.head(15)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Country,Wind Speed,Date,Hotel Name
0,35,busselton,-33.65,115.33,73.99,44,0,AU,14.41,1605948176,Busselton
1,80,arlit,18.74,7.39,72.97,21,0,NE,2.77,1605948185,Arlit
2,101,abu dhabi,24.60,72.70,70.45,22,0,IN,6.73,1605948190,Abu Road
3,152,banda aceh,25.48,80.33,78.49,19,0,IN,11.83,1605948201,Banda
4,228,bela,25.93,81.98,77.88,22,0,IN,11.99,1605948218,Bela Pratapgarh
5,282,oussouye,12.48,-16.55,73.40,94,0,SN,2.24,1605948230,Oussouye
6,401,natitingou,10.30,1.38,78.80,27,0,BJ,9.17,1605948258,Natitingou
7,450,riyadh,24.69,46.72,78.80,26,0,SA,14.99,1605947993,Riyadh
8,495,hermosillo,29.07,-110.97,73.00,36,0,MX,1.01,1605948085,Hermosillo
9,506,coahuayana,18.73,-103.68,74.71,80,0,MX,3.36,1605948280,Coahuayana de Hidalgo


In [12]:
google_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [13]:
for index, row in hotel_df.iterrows():
    # extract the city name, latitude, longtitude from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dictionary
    params["location"] = f"{lat},{lng}"
    
 # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(google_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("-------------------------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("--------------------------------------------End of Search----------------------------------------")

Retrieving Results for Index 0: hermanus.
Closest hotel in hermanus is Misty Waves Boutique Hotel.
-------------------------------
Retrieving Results for Index 1: new norfolk.
Missing field/result... skipping.
-------------------------------
Retrieving Results for Index 2: mataura.
Closest hotel in mataura is Ellie's Villa.
-------------------------------
Retrieving Results for Index 3: noumea.
Closest hotel in noumea is Nouvata.
-------------------------------
Retrieving Results for Index 4: cabo san lucas.
Closest hotel in cabo san lucas is Hotel Intercity Suape Costa Dourada.
-------------------------------
Retrieving Results for Index 5: de-kastri.
Closest hotel in de-kastri is Somon Hotel.
-------------------------------
Retrieving Results for Index 6: ancud.
Closest hotel in ancud is Panamericana Hotel Ancud.
-------------------------------
Retrieving Results for Index 7: merauke.
Missing field/result... skipping.
-------------------------------
Retrieving Results for Index 8: ky

Retrieving Results for Index 68: constitucion.
Closest hotel in constitucion is Bed & Breakfast Sweet Dreams.
-------------------------------
Retrieving Results for Index 69: bredasdorp.
Closest hotel in bredasdorp is Firlane House.
-------------------------------
Retrieving Results for Index 70: punta arenas.
Missing field/result... skipping.
-------------------------------
Retrieving Results for Index 71: melfi.
Closest hotel in melfi is Grand Hotel Garden.
-------------------------------
Retrieving Results for Index 72: saskylakh.
Closest hotel in saskylakh is Sberbank Rossii.
-------------------------------
Retrieving Results for Index 73: alice springs.
Closest hotel in alice springs is Hampton Inn Alice.
-------------------------------
Retrieving Results for Index 74: kapaa.
Closest hotel in kapaa is Hilton Garden Inn Kauai Wailua Bay.
-------------------------------
Retrieving Results for Index 75: emba.
Missing field/result... skipping.
-------------------------------
Retrievin

Retrieving Results for Index 134: tateyama.
Closest hotel in tateyama is Kagamigaura Onsen Rokuza.
-------------------------------
Retrieving Results for Index 135: tongzi.
Closest hotel in tongzi is Hengxu International Hotel.
-------------------------------
Retrieving Results for Index 136: tete.
Closest hotel in tete is Baobab Hotel Tete.
-------------------------------
Retrieving Results for Index 137: berbera.
Closest hotel in berbera is Damal Hotel.
-------------------------------
Retrieving Results for Index 138: avarua.
Closest hotel in avarua is The Rarotongan Beach Resort & Lagoonarium.
-------------------------------
Retrieving Results for Index 139: bengkulu.
Closest hotel in bengkulu is Mercure Bengkulu.
-------------------------------
Retrieving Results for Index 140: georgetown.
Closest hotel in georgetown is Macalister Mansion.
-------------------------------
Retrieving Results for Index 141: karavukovo.
Missing field/result... skipping.
-------------------------------


Closest hotel in nome is Aurora Inn & Suites.
-------------------------------
Retrieving Results for Index 200: naryan-mar.
Closest hotel in naryan-mar is Печора.
-------------------------------
Retrieving Results for Index 201: kieta.
Missing field/result... skipping.
-------------------------------
Retrieving Results for Index 202: necochea.
Closest hotel in necochea is Hotel Ñikén Spa & Business Center.
-------------------------------
Retrieving Results for Index 203: chokurdakh.
Closest hotel in chokurdakh is Sberbank Rossii.
-------------------------------
Retrieving Results for Index 204: caravelas.
Closest hotel in caravelas is Pousada Encanto Abrolhos.
-------------------------------
Retrieving Results for Index 205: xai-xai.
Missing field/result... skipping.
-------------------------------
Retrieving Results for Index 206: kavieng.
Closest hotel in kavieng is Nusa Island Retreat.
-------------------------------
Retrieving Results for Index 207: ishigaki.
Closest hotel in ishig

Retrieving Results for Index 266: salmon arm.
Closest hotel in salmon arm is Stagecoach Inn.
-------------------------------
Retrieving Results for Index 267: taoudenni.
Missing field/result... skipping.
-------------------------------
Retrieving Results for Index 268: luderitz.
Closest hotel in luderitz is LÜDERITZ NEST HOTEL.
-------------------------------
Retrieving Results for Index 269: villa union.
Closest hotel in villa union is Hotel Arisch.
-------------------------------
Retrieving Results for Index 270: buala.
Closest hotel in buala is Ghaseali Island Resort.
-------------------------------
Retrieving Results for Index 271: beira.
Closest hotel in beira is Beira Terrace Hotel.
-------------------------------
Retrieving Results for Index 272: yashkino.
Closest hotel in yashkino is Gostinitsa.
-------------------------------
Retrieving Results for Index 273: ostrovnoy.
Closest hotel in ostrovnoy is Sberbank Rossii.
-------------------------------
Retrieving Results for Index 

Retrieving Results for Index 332: sorland.
Closest hotel in sorland is Lofoten Værøy Brygge.
-------------------------------
Retrieving Results for Index 333: bathsheba.
Closest hotel in bathsheba is Atlantis Hotel.
-------------------------------
Retrieving Results for Index 334: asfi.
Closest hotel in asfi is RIAD LE CHEVAL BLANC - HÔTEL.
-------------------------------
Retrieving Results for Index 335: ibimirim.
Closest hotel in ibimirim is Hotel Paulistão.
-------------------------------
Retrieving Results for Index 336: rocha.
Closest hotel in rocha is Casa Rodo 53.
-------------------------------
Retrieving Results for Index 337: tiznit.
Missing field/result... skipping.
-------------------------------
Retrieving Results for Index 338: ramtek.
Closest hotel in ramtek is SPOT ON 41177 Hotel Manama.
-------------------------------
Retrieving Results for Index 339: ussel.
Closest hotel in ussel is Le Grand Hôtel à Ussel.
-------------------------------
Retrieving Results for Index 3

Retrieving Results for Index 399: karatuzskoye.
Missing field/result... skipping.
-------------------------------
Retrieving Results for Index 400: gatesville.
Closest hotel in gatesville is Holiday Inn Express & Suites Gatesville - N. Ft Hood.
-------------------------------
Retrieving Results for Index 401: natitingou.
Closest hotel in natitingou is Hotel Totora.
-------------------------------
Retrieving Results for Index 402: auki.
Closest hotel in auki is Auki Lodge.
-------------------------------
Retrieving Results for Index 403: union.
Closest hotel in union is Watt Hotel Rahway, Tapestry Collection by Hilton.
-------------------------------
Retrieving Results for Index 404: banfora.
Closest hotel in banfora is HOTEL CASCADES PALACE.
-------------------------------
Retrieving Results for Index 405: port-gentil.
Closest hotel in port-gentil is Hotel du Parc.
-------------------------------
Retrieving Results for Index 406: ternate.
Closest hotel in ternate is Grand Dafam Bela Te

Retrieving Results for Index 464: porto-vecchio.
Closest hotel in porto-vecchio is Modern'Hotel.
-------------------------------
Retrieving Results for Index 465: lagoa.
Closest hotel in lagoa is Graciosa Resort - Biosphere Island Hotel.
-------------------------------
Retrieving Results for Index 466: san pedro de ycuamandiyu.
Closest hotel in san pedro de ycuamandiyu is Hotel Santoro.
-------------------------------
Retrieving Results for Index 467: barcelos.
Closest hotel in barcelos is Hotel Bagoeira.
-------------------------------
Retrieving Results for Index 468: komnina.
Missing field/result... skipping.
-------------------------------
Retrieving Results for Index 469: san jose iturbide.
Closest hotel in san jose iturbide is Hotel Santoro.
-------------------------------
Retrieving Results for Index 470: puerto del rosario.
Closest hotel in puerto del rosario is Pestana.
-------------------------------
Retrieving Results for Index 471: mosquera.
Closest hotel in mosquera is Hot

In [17]:
hotel_df.head(20)

,City,Country,Lat,Lng,Max Temp,Hotel Name
0,hermanus,ZA,-34.42,19.23,73.99,Misty Waves Boutique Hotel
1,new norfolk,LR,6.65,-10.59,80.60,NaN
2,mataura,NZ,-46.19,168.86,48.99,Ellie's Villa
3,noumea,NC,-22.28,166.46,73.40,Nouvata
4,cabo san lucas,BR,-8.28,-35.03,77.00,Hotel Intercity Suape Costa Dourada
5,de-kastri,RU,51.47,140.78,26.46,Somon Hotel
6,ancud,CL,-41.87,-73.82,54.00,Panamericana Hotel Ancud
7,merauke,ID,-8.47,140.33,84.74,NaN
8,kyzyl-suu,KG,42.34,78.01,33.39,guesthouse Smiling Aida
9,nikolskoye,RU,59.70,30.79,34.00,NaN


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))